In [ ]:
import numpy as np
import cvxpy as cp
import pandas as pd

from qutip import (
    coherent,
    ket2dm,
    Qobj,
    fidelity,
    fock,

)

import time
from math import ceil
import jax

import jax.numpy as jnp
import equinox as eq
import dynamiqs as dq


jax.config.update("jax_enable_x64", False)
jax.config.update("jax_platform_name", "cpu")
# enable compilation cache
jax.config.update("jax_compilation_cache_dir", "/tmp/jax_cache")


class Displacer(eq.Module):
    evals: jnp.ndarray
    evecs: jnp.ndarray
    range: jnp.ndarray
    t_scale: jnp.ndarray
    a: jnp.ndarray

    def __init__(self, n):
        # The off-diagonal of the real-symmetric similar matrix T.
        sym = (2 * (jnp.arange(1, n) % 2) - 1) * jnp.sqrt(jnp.arange(1, n))
        # Solve the eigensystem.

        # construct a tri-diagonal matrix
        _mat = jnp.diag(sym, 1) + jnp.diag(sym, -1)

        self.evals, self.evecs = jax.scipy.linalg.eigh(_mat)
        self.range = np.arange(n)
        self.t_scale = 1j ** (self.range % 2)

        self.a = dq.destroy(n) #这个生成的数据变量是SparseDIAQArray
        self.a = jnp.asarray(np.array(self.a))

    @jax.jit
    def new_method(self, alpha):
        # Diagonal of the transformation matrix P, and apply to eigenvectors.
        r, theta = jnp.abs(alpha), jnp.angle(alpha)
        transform = self.t_scale * (jnp.exp(1j * theta)) ** -self.range
        evecs = transform[:, None] * self.evecs
        # Get the exponentiated diagonal.
        diag = jnp.exp(1j * r * self.evals)
        return jnp.conj(evecs) @ (diag[:, None] * evecs.T)

    # @jax.jit
    @eq.filter_jit
    def old_method(self, alpha_single):
        return jax.scipy.linalg.expm(
            alpha_single * self.a.conj().T - alpha_single.conj() * self.a
            # alpha_single * self.a.dag() - alpha_single.conj() * self.a
        )


In [ ]:
num_iterations = 3  # 循环次数
z1 = np.zeros(num_iterations)
z2 = np.zeros(num_iterations)
z3 = np.zeros(num_iterations)
z4 = np.zeros(num_iterations)
z5 = np.zeros(num_iterations)
z0 = np.zeros(num_iterations)


for p in range(num_iterations):
    start_time1 = time.time()  # 记录开始时间

    N = 30   # 这个是N的值，即# Fock space dimension
    beta_max = 4  # 这个是alpha_max的值，即Maximum coherent state amplitude
    nm = 20  # 网格数量，即number of probes in the x/p direction
    trunction = 2 # 截断倍数

    # 猫态构建
    # Coherent state amplitudes for test cat state
    alpha_range = 2
    alphas = np.array([alpha_range, -alpha_range])

    # 初始化类
    displace_op = Displacer(ceil(N * trunction))
    state_fock0 = fock(ceil(N * trunction), 0)
    state_fock0 = jnp.array(state_fock0.full())


    # Test-state
    psi = sum([coherent(N, a) for a in alphas])
    psi = psi.unit()  # 归一化
    rho = ket2dm(psi)  # 将态矢量转换为密度矩阵

    # 以上构建了这样一个猫态

    # alpha_max = 5  # setting the limits on the Wigner plot
    # 似乎是无用数据

    start_time3 = time.time()  # 记录开始时间，以下四个版块都算在A构建里面
    # 基底构建
    # construct vectors
    basis_vectors = []
    for i in range(N):
        vector = fock(N, i)
        basis_vectors.append(jnp.real(vector.full()))

    # construct operators
    basis_dms = []
    for vector1 in basis_vectors:
        for vector2 in basis_vectors:
            dm = jnp.outer(vector1, vector2)
            basis_dms.append(dm)

    # 和ideal的homodyne一样，这里是构建了一组（N*N个）由占据数表象构成的基底，就是|n><n|


    # 相干态构建
    xvec_betas = jnp.linspace(-beta_max, beta_max, nm)  # 范围加上数量
    yvec_betas = jnp.linspace(-beta_max, beta_max, nm)
    X, Y = jnp.meshgrid(xvec_betas, yvec_betas)  # 生成网格
    # flatten the grid of probe states into a 1D array
    betas = (X + 1j * Y).ravel()
    # X为实数部分，Y为虚数部分，并且将二维的数组展平成一维的数组,接下来画图与构建都会用到

    # 测量算符构建
    # Pis = Parallel(n_jobs=16, verbose=5)(delayed(qfunc_ops)(N,beta) for beta in betas)

    betas_vec = betas.flatten(order="F")
    # betas_vec = betas.reshape(400,1)
    # betas_vec = jnp.array(betas_vec)
    print(betas.shape)


    # @jax.jit #会变快
    def _coherent_dm(beta):
        op = displace_op.old_method(beta)
        op = op @ state_fock0
        op = op @ op.T.conj()
        # op = op @ op.dag()

        op = op[:N, :N]  # truncate to wanted Hilbert space size
        # 不用qutip的quantum object
        # 上面做了一个截断，将矩阵截断至hilbert_size行与hilbert_size列上,注意截断之后数据类型错误，要补一个Qobj操作

        return op


    Pis = jax.vmap(_coherent_dm)(betas_vec)
    # Pis = jax.vmap(qfunc_ops_fixed)(betas_vec)


    def func(index):
        return jnp.trace(
            Pis[index[0]] @ basis_dms[index[1]]
        )  # @表示矩阵乘法，里面的index[0],[1]表示index的第一项与第二项，结合后面就是行与列


    A = np.zeros((len(Pis), N * N), dtype=np.complex64)
    # A_p = Parallel(n_jobs=16, verbose=5, backend="multiprocessing")(delayed(func)(idx,a) for idx, a in np.ndenumerate(A))

    # A_2 = np.zeros((len(Pis), N*N,N*N),dtype=np.complex64)
    A_2 = []
    for (i, j), value in np.ndenumerate(A):
        A_2.append([i, j])

    basis_dms = jnp.array(basis_dms)
    A_2 = jnp.array(A_2)
    # A_p = jax.vmap(func)(A_2)

    A_p = jax.lax.map(func,A_2,batch_size=100)


    # A_p = Parallel(n_jobs=16, verbose=5)(delayed(func)(idx,a) for idx, a in np.ndenumerate(A))# 后面那一个可以视为一个整体，见笔记


    A = np.reshape(A_p, (len(Pis), N * N))

    end_time3 = time.time()


    # Pis = Qobj(Pis)
    # 构建理想状态的b
    start_time4 = time.time()  # 记录开始时间
    # b = expect(Pis, rho) #算期望，Tr[A_tho]的形式，前者是态，后者是密度矩阵
    b = jnp.trace(Pis @ rho.full(), axis1=1, axis2=2)
    b = np.array(b)  # 不知道这一段代码是来干什么的，删了似乎也没事
    end_time4 = time.time()

    # 凸优化
    start_time2 = time.time()  # 记录开始时间
    X = cp.Variable((N, N), hermitian=True)
    # 构建待优化的矩阵
    cost = cp.norm(A @ cp.vec(X, order="F") - b, 2)
    # 构建cost函数
    constraints = [cp.trace(X) == 1, X >> 0]
    # 构建约束条件
    prob = cp.Problem(cp.Minimize(cost), constraints)
    prob.solve(
        # solver=cp.SCS,
        #        verbose=True,
        #        mkl=True,
               )
    # 求解问题
    end_time2 = time.time()

    # 保真度
    reconstructed_rho = Qobj(X.value)


    fidelity(reconstructed_rho, Qobj(rho))


    # 处理与结果输出
    end_time1 = time.time()  # 记录结束时间
    elapsed_time1 = end_time1 - start_time1  # 计算所用时间
    elapsed_time2 = end_time2 - start_time2  # 计算所用时间
    elapsed_time3 = end_time3 - start_time3  # 计算所用时间
    elapsed_time4 = end_time4 - start_time4  # 计算所用时间
    z0[p] = fidelity(reconstructed_rho, Qobj(rho))  # 保真度
    z1[p] = elapsed_time1
    z2[p] = elapsed_time2
    z3[p] = elapsed_time3
    z4[p] = elapsed_time4
    # elapsed_time5 = elapsed_time3 - elapsed_time4
    # 自变量
    # z5[p] = N
    # print("迭代的N =",N )

    z5[p] = N
    print("迭代的相空间极限 =", beta_max)

    print(f"总迭代时间 Iteration {p + 1}: {elapsed_time1:.4f} seconds")  # 构建A的时间
    print(f"优化时间 Iteration {p + 1}: {elapsed_time2:.4f} seconds")  # cvx时间
    print(f"矩阵时间 Iteration {p + 1}: {elapsed_time3:.4f} seconds")
    print(f"构建b的时间 Iteration {p + 1}: {elapsed_time4:.4f} seconds")
    # print(f"总迭代时间减去构建b的时间 Iteration {p + 1}: {elapsed_time5:.4f} seconds")

    print("保真度", p + 1, z0[p])


# 数据写出
# 创建数据
data = [z5, z0, z1, z2, z3, z4]

# 转换为 DataFrame
df = pd.DataFrame(data)

# 将数据写入 Excel 文件
df.to_excel("time_N_cvx_ideal_heterodyne.xlsx", index=False)

print("数据已成功写入 time_N_cvx_ideal_heterodyne.xlsx")